In [ ]:
import torch
import torch.nn as nn
import math

In [ ]:
class LoRALinear(nn.Module):
    def __init__(self, in_features, out_features, r=4, lora_alpha=1):
        super().__init__()
        # 1. 原来的全连接层 (模拟冻结的预训练权重)
        # weight 的形状是 [out_features, in_features]
        self.pretrained = nn.Linear(in_features, out_features, bias=False)#提取预训练的线性层

        # --- 核心：冻结它！不许更新梯度 ---
        self.pretrained.weight.requires_grad = False 
        
        # 2. LoRA 的两个小矩阵 A 和 B
        # A: 降维 [r, in_features]
        # B: 升维 [out_features, r]
        self.lora_A = nn.Parameter(torch.randn(r, in_features))
        self.lora_B = nn.Parameter(torch.zeros(out_features, r)) #parameter把后面的东西变成要训练的参数，当你用 nn.Parameter(...) 创建它们时，PyTorch 默认会把它们的 requires_grad 设为 True。我们没有去改这个默认设置，所以它们是可训练的。
        
        # 初始化技巧：B 初始化为 0，这样刚开始训练时，LoRA 的影响是 0
        # 保证模型一开始的表现和预训练模型完全一样
        nn.init.kaiming_uniform_(self.lora_A, a=math.sqrt(5))
        nn.init.zeros_(self.lora_B)
        
        self.scaling = lora_alpha / r

    def forward(self, x):
        # x 的形状: [batch_size, in_features]
        
        # 路径 1: 走原始模型 (Pretrained)
        # result_base = x @ W.T
        result_base = self.pretrained(x)
        
        # 路径 2: 走 LoRA 分支 (Adapter)
        # 公式: x @ A.T @ B.T * scaling
        # 也就是先把 x 降维到 r，再升维回 out_features
        lora_process = (x @ self.lora_A.T) @ self.lora_B.T * self.scaling
        
        # 最终结果 = 原始结果 + 增量
        return result_base + lora_process


In [ ]:
# --- 动手测试 ---
# 假设我们有一个巨大的层：输入 1024 维，输出 1024 维
layer = LoRALinear(1024, 1024, r=8)

# 打印参数量对比
total_params = sum(p.numel() for p in layer.parameters())
trainable_params = sum(p.numel() for p in layer.parameters() if p.requires_grad)

print(f"原始权重 W 参数量: {1024*1024}")
print(f"LoRA (A+B) 参数量: {trainable_params}")
print(f"节省了: {(1 - trainable_params/(1024*1024))*100:.2f}% 的显存！")

# 测试一下前向传播
input_data = torch.randn(1, 1024)#生成一个形状为 (1, 1024) 的张量（Tensor），里面填满了符合标准正态分布的随机数
output = layer(input_data)
print("输出维度:", output.shape) # 应该是 [1, 1024]，维度没变！